# Exercise 3


In [1]:
# pip install tabulate

In [2]:
# Download latest FFmpeg static build.  
exist = !which ffmpeg
if not exist:
  !curl https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz -o ffmpeg.tar.xz \
     && tar -xf ffmpeg.tar.xz && rm ffmpeg.tar.xz
  ffmdir = !find . -iname ffmpeg-*-static
  path = %env PATH
  path = path + ':' + ffmdir[0]
  %env PATH $path

!which ffmpeg

/usr/bin/ffmpeg


In [3]:
# import some modules
import os
import subprocess

## Examination with ffprobe

In [4]:
# load files
file_paths = [
    "./Voyage_to_the_Planet_of_Prehistoric_Women.mp4",
    "./The_Gun_and_the_Pulpit.avi",
    "./Last_man_on_earth_1964.mov",
    "./Cosmos_War_of_the_Planets.mp4",
    "./The_Hill_Gang_Rides_Again.mp4"
]

In [5]:
# display the videos
from IPython.display import Video

for file_path in file_paths: 
    display(Video(f"{file_path}"))

In [6]:
# ffprobe examination code
def run_ffprobe(file_path):
    # commands to probe video
    cmd_video = [
        'ffprobe','-v', 'error',
        '-hide_banner','-select_streams', 'v',
        '-print_format', 'default', '-show_entries',
        'stream=codec_name, width, height, display_aspect_ratio, r_frame_rate, bit_rate',
        file_path
    ]
    
    # commands to probe audio
    cmd_audio = [
        'ffprobe','-v', 'error',
        '-hide_banner','-select_streams', 'a',
        '-print_format', 'default', '-show_entries',
        'stream=codec_name, bit_rate, channel_layout',
        file_path
    ]
    
    try:
        # extract video format from file_path
        video_format = file_path.split('.')[-1]
        
        # use subprocess
        result_video = subprocess.check_output(cmd_video, universal_newlines=True)
        result_audio = subprocess.check_output(cmd_audio, universal_newlines=True)
        
        # store video information
        video_info = {}
        for line in result_video.split('\n'):
            if '=' not in line:
                continue
            key, value = line.split('=')
            key = key.strip()
            value = value.strip()
            
            # Convert framerate to float
            if key == 'r_frame_rate':
                num, denom = value.split('/')
                value = float(num) / float(denom)
                value = format(value,'.1f')
                
            # convert video bitrate to Mb/s
            video_info[key] = value
            if key == 'bit_rate':
                bitrate = float(value) /1048576
                video_info[key] = format(float(bitrate), '.1f')

        # store audio information
        audio_info = {}
        for line in result_audio.split('\n'):
            if '=' not in line:
                continue
            key, value = line.split('=')
            audio_info[key.strip()] = value.strip()
            
            # convert audio bitrate to Kb/s
            if key == 'bit_rate':
                bitrate = float(value) / 8000  # Convert to kb/s
                audio_info[key] = format(float(bitrate), '.1f')

        # this will later be stored in file info dictionary
        return {
            'video_format': video_format,
            'video_info': video_info,
            'audio_info': audio_info
        }
        
    # error Handling
    except subprocess.CalledProcessError as e:
        print(f"Failed to read {file_path}")
        return None

# use ffprobe function on the audio files
file_info = {}
for file_path in file_paths:
    result = run_ffprobe(file_path)
    if result:
        file_info[os.path.basename(file_path)] = result

# Print information of the files
for file_name, info in file_info.items():
    print(f"File: {file_name}")
    print(f'video_format: {info["video_format"]}')
    print("Video info:")
    for key, value in info['video_info'].items():
        print(f"  {key}: {value}")
    print("Audio info:")
    for key, value in info['audio_info'].items():
        print(f"  {key}: {value}")
    print('\n', '-' * 60)


File: Voyage_to_the_Planet_of_Prehistoric_Women.mp4
video_format: mp4
Video info:
  codec_name: hevc
  width: 640
  height: 360
  display_aspect_ratio: 16:9
  r_frame_rate: 30.0
  bit_rate: 7.7
Audio info:
  codec_name: mp3
  channel_layout: stereo
  bit_rate: 40.0

 ------------------------------------------------------------
File: The_Gun_and_the_Pulpit.avi
video_format: avi
Video info:
  codec_name: rawvideo
  width: 720
  height: 404
  display_aspect_ratio: 0:1
  r_frame_rate: 25.0
  bit_rate: 83.4
Audio info:
  codec_name: pcm_s16le
  channel_layout: unknown
  bit_rate: 192.0

 ------------------------------------------------------------
File: Last_man_on_earth_1964.mov
video_format: mov
Video info:
  codec_name: prores
  width: 640
  height: 360
  display_aspect_ratio: 16:9
  r_frame_rate: 24.0
  bit_rate: 8.9
Audio info:
  codec_name: pcm_s16le
  channel_layout: stereo
  bit_rate: 192.0

 ------------------------------------------------------------
File: Cosmos_War_of_the_Planet

## Text reports

### Format requirements:
- Video format (container): mp4 
- Video codec: h.264 
- Audio codec: aac 
- Frame rate: 25 FPS 
- Aspect ratio: 16:9  
- Resolution: 640 x 360 
- Video bit rate: 2 – 5 Mb/s 
- Audio bit rate: up to 256 kb/s 
- Audio channels: stereo 

In [7]:
# function to compare formats
def compare_formats(video_format, video_info, audio_info):
    
    # the expected format criteria
    expected_format = {
        "container": "mp4",
        "video_codec": "h264",
        "audio_codec": "aac",
        "frame_rate": 25.0,
        "aspect_ratio": "16:9",
        "resolution": (640, 360),
        "video_bit_rate": (2, 5),  # 2 - 5 Mb/s
        "audio_bit_rate": 256,  # up to 256 kb/s
        "audio_channels": "stereo"
    }

    # dictionary to store the comparison results
    comparison_result = {}

    # compare video format
    if video_format != expected_format["container"]:
        comparison_result["container"] = (video_format, expected_format["container"])
        
    # compare video codec
    if video_info["codec_name"] != expected_format["video_codec"]:
        comparison_result["video_codec"] = (video_info["codec_name"], expected_format["video_codec"])

    # compare audio codec
    if audio_info["codec_name"] != expected_format["audio_codec"]:
        comparison_result["audio_codec"] = (audio_info["codec_name"], expected_format["audio_codec"])

    # compare frame rate
    if float(video_info["r_frame_rate"]) != float(expected_format["frame_rate"]):
        comparison_result["frame_rate"] = (video_info["r_frame_rate"], expected_format["frame_rate"])

    # compare aspect ratio
    if video_info["display_aspect_ratio"] != expected_format["aspect_ratio"]:
        comparison_result["aspect_ratio"] = (video_info["display_aspect_ratio"], expected_format["aspect_ratio"])

    # compare resolution
    if (int(video_info["width"]), int(video_info["height"])) != expected_format["resolution"]:
        comparison_result["resolution"] = ((int(video_info["width"]), int(video_info["height"])), expected_format["resolution"])

    # compare video bit rate
    if float(video_info["bit_rate"]) < expected_format["video_bit_rate"][0] or float(video_info["bit_rate"]) > expected_format["video_bit_rate"][1]:
        comparison_result["video_bit_rate"] = (float(video_info["bit_rate"]), f"{expected_format['video_bit_rate'][0]} - {expected_format['video_bit_rate'][1]} Mb/s")

    # compare audio bit rate
    if float(audio_info["bit_rate"]) > expected_format["audio_bit_rate"]:
        comparison_result["audio_bit_rate"] = (float(audio_info["bit_rate"]), f"up to {expected_format['audio_bit_rate']} kb/s")

    # compare audio channels
    if audio_info["channel_layout"] != expected_format["audio_channels"]:
        comparison_result["audio_channels"] = (audio_info["channel_layout"], expected_format["audio_channels"])

    
    return comparison_result

# usage on the file paths
for file_name, info in file_info.items():
    video_format = info["video_format"]
    video_info = info["video_info"]
    audio_info = info["audio_info"]
    comparison_result = compare_formats(video_format, video_info, audio_info)
    print(f"File: {file_name}")
    print("Comparison Result:")
    for key, value in comparison_result.items():
        print(f"  {key}: {value[0]} || requirement: {value[1]}")
    print('\n', '-' * 60)


File: Voyage_to_the_Planet_of_Prehistoric_Women.mp4
Comparison Result:
  video_codec: hevc || requirement: h264
  audio_codec: mp3 || requirement: aac
  frame_rate: 30.0 || requirement: 25.0
  video_bit_rate: 7.7 || requirement: 2 - 5 Mb/s

 ------------------------------------------------------------
File: The_Gun_and_the_Pulpit.avi
Comparison Result:
  container: avi || requirement: mp4
  video_codec: rawvideo || requirement: h264
  audio_codec: pcm_s16le || requirement: aac
  aspect_ratio: 0:1 || requirement: 16:9
  resolution: (720, 404) || requirement: (640, 360)
  video_bit_rate: 83.4 || requirement: 2 - 5 Mb/s
  audio_channels: unknown || requirement: stereo

 ------------------------------------------------------------
File: Last_man_on_earth_1964.mov
Comparison Result:
  container: mov || requirement: mp4
  video_codec: prores || requirement: h264
  audio_codec: pcm_s16le || requirement: aac
  frame_rate: 24.0 || requirement: 25.0
  video_bit_rate: 8.9 || requirement: 2 - 5 Mb

In [8]:
# make report txt
def generate_report(file_path, video_file_path, comparison_result):
    with open(file_path, 'a') as f:
        # create report format
        f.write(f"Comparison Result for {video_file_path}:\n")
        # iterate through the comparison results and print them in the txt file
        for key, value in comparison_result.items():
            f.write(f"  {key}: {value[0]} || requirement: {value[1]}\n")
        f.write("-" * 60 + "\n")  

# Create or edit the report.txt
report_file_name = "report.txt"
with open(report_file_name, 'w') as f:
    f.write("Comparison Results\n")
    f.write("=" * 60 + "\n")

# use generate_report with information from comparison_result
for file_name, info in file_info.items():
    video_format = info["video_format"]
    video_info = info["video_info"]
    audio_info = info["audio_info"]
    comparison_result = compare_formats(video_format, video_info, audio_info)
    generate_report(report_file_name, file_name, comparison_result)

print(f"Report generated: {report_file_name}")


Report generated: report.txt


## Convert with ffmpeg

### Format requirements:
- Video format (container): mp4 
- Video codec: h.264 
- Audio codec: aac 
- Frame rate: 25 FPS 
- Aspect ratio: 16:9  
- Resolution: 640 x 360 
- Video bit rate: 2 – 5 Mb/s 
- Audio bit rate: up to 256 kb/s 
- Audio channels: stereo 

In [9]:
# formatting the videos with ffmpeg
def change_format(file_path):
    # file name format for new files
    output_file = os.path.splitext(file_path)[0] + "_formatOK.mp4"

    # FFmpeg command to convert the file
    cmd_convert = [
        'ffmpeg',
        '-i', file_path,
        '-c:v', 'libx264', # h264
        '-vf', 'scale=640:360',
        '-aspect', '16:9',  
        '-r', '25',  # framerate
        '-b:v', '3000000', # video bitrate
        '-c:a', 'aac', # audio codec
        '-b:a', '1800000',  # audio bitrate
        '-ac', '2', # stereo
        output_file  
    ]

    try:
        # use subprocess to execute
        subprocess.check_output(cmd_convert, stderr=subprocess.STDOUT, universal_newlines=True)
        print(f"Converted {file_path} to {output_file}")
        
    # error Handling
    except subprocess.CalledProcessError as e:
        print(f"Failed to convert {file_path}: {e}")
        return None

# Convert each file in file_paths (on the top)
for file_path in file_paths:
    change_format(file_path)


Converted ./Voyage_to_the_Planet_of_Prehistoric_Women.mp4 to ./Voyage_to_the_Planet_of_Prehistoric_Women_formatOK.mp4
Converted ./The_Gun_and_the_Pulpit.avi to ./The_Gun_and_the_Pulpit_formatOK.mp4
Converted ./Last_man_on_earth_1964.mov to ./Last_man_on_earth_1964_formatOK.mp4
Converted ./Cosmos_War_of_the_Planets.mp4 to ./Cosmos_War_of_the_Planets_formatOK.mp4
Converted ./The_Hill_Gang_Rides_Again.mp4 to ./The_Hill_Gang_Rides_Again_formatOK.mp4


In [10]:
# load new copnverted files
file_paths_new = [
    "./Voyage_to_the_Planet_of_Prehistoric_Women_formatOK.mp4",
    "./The_Gun_and_the_Pulpit_formatOK.mp4",
    "./Last_man_on_earth_1964_formatOK.mp4",
    "./Cosmos_War_of_the_Planets_formatOK.mp4",
    "./The_Hill_Gang_Rides_Again_formatOK.mp4"
]


# use ffprobe function on the audio files
file_info = {}
for file_path in file_paths_new:
    result = run_ffprobe(file_path)
    if result:
        file_info[os.path.basename(file_path)] = result

# Print information of the new files
for file_name, info in file_info.items():
    print(f"File: {file_name}")
    print(f'video_format: {info["video_format"]}')
    print("Video info:")
    for key, value in info['video_info'].items():
        print(f"  {key}: {value}")
    print("Audio info:")
    for key, value in info['audio_info'].items():
        print(f"  {key}: {value}")
    print('\n', '-' * 60)   

File: Voyage_to_the_Planet_of_Prehistoric_Women_formatOK.mp4
video_format: mp4
Video info:
  codec_name: h264
  width: 640
  height: 360
  display_aspect_ratio: 16:9
  r_frame_rate: 25.0
  bit_rate: 2.7
Audio info:
  codec_name: aac
  channel_layout: stereo
  bit_rate: 32.1

 ------------------------------------------------------------
File: The_Gun_and_the_Pulpit_formatOK.mp4
video_format: mp4
Video info:
  codec_name: h264
  width: 640
  height: 360
  display_aspect_ratio: 16:9
  r_frame_rate: 25.0
  bit_rate: 2.8
Audio info:
  codec_name: aac
  channel_layout: stereo
  bit_rate: 33.9

 ------------------------------------------------------------
File: Last_man_on_earth_1964_formatOK.mp4
video_format: mp4
Video info:
  codec_name: h264
  width: 640
  height: 360
  display_aspect_ratio: 16:9
  r_frame_rate: 25.0
  bit_rate: 2.9
Audio info:
  codec_name: aac
  channel_layout: stereo
  bit_rate: 38.2

 ------------------------------------------------------------
File: Cosmos_War_of_the_

In [11]:
# display the formatted videos
from IPython.display import Video

for file_path in file_paths_new: 
    display(Video(f"{file_path}"))